<img src="img/logo.png">

# Algorithme de détection des vrais / faux billets

In [1]:
import pandas as pd
import pickle
from mylib.pickle import read_pickle
import os

## Récupération du modèle

In [2]:
# Chemin absolu du dossier actuel
current_dir = os.getcwd()

# Chemin relatif vers le modèle
relative_path = 'p10_lr1'

# Chemin complet
model_path = os.path.join(current_dir, relative_path)

# Extraction du modèle
pickle_lr = read_pickle(model_path, '')

## Création de l'algorithme

In [3]:
def pred_billets(path_data, model):
    """
    Fait des prédictions sur les données de billets en utilisant un modèle donné

    Args:
        data (DataFrame): les données de billets à prédire
        model: le modèle utilisé pour les prédictions

    Returns:
        DataFrame: Un DataFrame contenant les prédictions et les probabilités de véracité
    """
    # Chargement des données de billets depuis le fichier CSV
    data = pd.read_csv(path_data, sep=",")
    
    # Sélection des colonnes pertinentes et définition de la variable 'id' comme index
    data = data[['id', 'length', 'margin_low', 'margin_up', 'height_right']].set_index('id')
    
    # Utilisation du modèle pour faire des prédictions
    predictions = model.predict(data)
    probabilities = model.predict_proba(data)[:, 1]  # Probabilités des vrais billets
    
    # Ajout des résultats au DataFrame
    data["véracité du billet"] = predictions
    data['probabilité de véracité'] = probabilities.round(4)
    
    # Affichage des ID des vrais billets
    billets_vrais = data[data["véracité du billet"] == 1].index.tolist()
    print('=' * 45, "\nIdentifiants des vrais billets :", ', '.join(billets_vrais))
    
    # Affichage des ID des faux billets
    billets_faux = data[data["véracité du billet"] == 0].index.tolist()
    print("Identifiants des faux billets :", ', '.join(billets_faux),'\n' + '=' * 45 + '\n')
    
    return data

## Détection des vrais / faux billets

### Test échantillon 1

In [7]:
path_data = 'src/billets_production.csv'
pred_billets(path_data, pickle_lr)

Identifiants des vrais billets : A_4, A_5
Identifiants des faux billets : A_1, A_2, A_3 



,length,margin_low,margin_up,height_right,véracité du billet,probabilité de véracité
id,,,,,,
A_1,111.42,5.21,3.30,103.54,False,0.0128
A_2,112.09,6.00,3.31,104.13,False,0.0009
A_3,111.57,4.99,3.39,104.29,False,0.0030
A_4,113.20,4.44,3.03,104.34,True,0.9304
A_5,113.33,3.77,3.16,103.56,True,0.9995


### Test échantillon 2

In [10]:
path_data = 'src/billets_test.csv'
data = pd.read_csv(path_data, sep=",")
data

,diagonal,height_left,height_right,margin_low,margin_up,length,id
0,172.09,103.95,103.73,4.39,3.09,113.19,B_1
1,171.52,104.17,104.03,5.27,3.16,111.82,B_2
2,171.78,103.80,103.75,3.81,3.24,113.39,B_3
3,172.02,104.08,103.99,5.57,3.30,111.10,B_4
4,171.79,104.34,104.37,5.00,3.07,111.87,B_5


In [11]:
pred_billets(path_data, pickle_lr)

Identifiants des vrais billets : B_1, B_3
Identifiants des faux billets : B_2, B_4, B_5 



,length,margin_low,margin_up,height_right,véracité du billet,probabilité de véracité
id,,,,,,
B_1,113.19,4.39,3.09,103.73,True,0.9892
B_2,111.82,5.27,3.16,104.03,False,0.0124
B_3,113.39,3.81,3.24,103.75,True,0.9988
B_4,111.10,5.57,3.30,103.99,False,0.0003
B_5,111.87,5.00,3.07,104.37,False,0.0176
